## Import a Bunch of Stuff

In [5]:
import codecs
import random
import math
import torch
import itertools
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle

use_cuda = torch.cuda.is_available()
plt.style.use('seaborn-whitegrid')
%matplotlib inline

## Define all the Classes & Functions

In [21]:
def load_data(fpath, label, max_story_length):
    data = []
    with codecs.open(fpath, 'r', 'utf-8', errors='ignore') as f:
        lines = f.readlines()
        for l in lines:
            l = l.rstrip()
            data.append((l.split(' ')[0:max_story_length], label))
    return data

class Net(nn.Module):
    def __init__(self, vocab_size, embd_size, out_chs, filter_heights):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embd_size)
        # nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, ...
        self.conv = nn.ModuleList([nn.Conv2d(1, out_chs, (fh, embd_size)) for fh in filter_heights])
        self.dropout = nn.Dropout(.5)
        self.fc1 = nn.Linear(out_chs*len(filter_heights), 1)
        
    def forward(self, x):
        x = self.embedding(x) # (N, seq_len, embd_dim)
        x = x.unsqueeze(1) # (N, Cin, W, embd_dim), insert Channnel-In dim
        # Conv2d
        #    Input : (N,Cin, Hin, Win )
        #    Output: (N,Cout,Hout,Wout) 
        # squeeze(3) means 2D to 1D; (N,Cout,Hout,Wout) -> [(N,Cout,Hout==seq_len)] * len(filter_heights)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.conv]
        # max_pool1d(input, kernel_size, ..
        # (N, Cout, seq_len) --(max_pool1d)--> (N, Cout, 1) --(squeeze(2))--> (N, Cout)
        # [(N, Cout)]  len(filter_heights)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1) # (N, Cout*len(filter_heights))
        x = self.dropout(x)
        x = self.fc1(x)
        probs = F.sigmoid(x)
        
        return probs

def train(model, data, batch_size, n_epoch):
    model.train() # Sets the module in training mode. This has any effect only on modules such as Dropout or BatchNorm.
    if use_cuda:
        model.cuda()
    losses = []
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    for epoch in range(n_epoch):
        epoch_loss = 0.0
        random.shuffle(data)
        for i in range(0, len(data)-batch_size, batch_size): # discard some last elements
            in_data, labels = [], []
            for sentence, label in data[i: i+batch_size]:
                index_vec = [w2i[w] for w in sentence]
                pad_len = max(0, max_sentence_len - len(index_vec))
                index_vec += [0] * pad_len
                index_vec = index_vec[:max_sentence_len] ## TBD for same len
                in_data.append(index_vec)
                labels.append(label)
            sent_var = Variable(torch.LongTensor(in_data))
            if use_cuda: sent_var = sent_var.cuda()

            target_var = Variable(torch.Tensor(labels).unsqueeze(1))
            if use_cuda: target_var = target_var.cuda()
            optimizer.zero_grad()
            probs = model(sent_var)
            loss = F.binary_cross_entropy(probs, target_var)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.data[0]
        #print('epoch: {:d}, loss: {:.3f}'.format(epoch, epoch_loss))
        losses.append(epoch_loss)
    #print('Training avg loss: {:.3f}'.format(sum(losses)/len(losses)))
        
    return model, losses

def test(model, test_data, min_sentence_len):
    model.eval()
    loss = 0
    run_scores = []
    for sentence, label in test_data:
        if len(sentence) < min_sentence_len:  # to short for CNN's filter
            continue
        index_vec = [w2i[w] for w in sentence]
        sent_var = Variable(torch.LongTensor([index_vec]))
        if use_cuda: sent_var = sent_var.cuda()
        out = model(sent_var)
        score = out.data[0][0].cpu().numpy()
        run_scores.append(score)
    
    return run_scores

def param_accuracy_graph(out_ch, embd_size, filters):
        
    batch_size = 256
    n_epoch = 25
    params = []
    accuracy = []
    
    model_num = 0
    total_models = len(out_ch)*len(embd_size)*len(filters)
    agg_scores = [0]*225
    
    for out_ch, embd_size, filters in itertools.product(out_ch, embd_size, filters):
        
        model_num = model_num + 1
        print("Model {}/{}  {:.1f}%".format(model_num, total_models, model_num*100/total_models))
        
        model = Net(vocab_size, embd_size, out_ch, filters)
        model, losses = train(model, train_data, batch_size, n_epoch)
        run_scores = test(model, test_data, max(filters))
        
        agg_scores = [sum(x) for x in zip(agg_scores, run_scores)]
        print(agg_scores)
    return agg_scores

## Load & Preprocess Data

In [13]:
human = load_data('./dataset/stories/fusionModel_humanStories2.txt', 1, 150)
machine = load_data('./dataset/stories/fusionModel_machineStories2.txt', 0, 150)
test_data = load_data('./dataset/stories/testStories.txt', 0, 150)
train_data = human + machine
data = human + machine + test_data
max_sentence_len = max([len(sentence) for sentence, _ in data])
print('sentence maxlen', max_sentence_len)

vocab = []
for d, _ in data:
    for w in d:
        if w not in vocab: vocab.append(w)
            
vocab = sorted(vocab)
vocab_size = len(vocab)
print('vocab examples:', vocab[:10])
print('vocab size', len(vocab))

w2i = {w:i for i,w in enumerate(vocab)}
i2w = {i:w for i,w in enumerate(vocab)}

# split data into train and test data

print('n_train', len(train_data))
print('n_test', len(test_data))

sentence maxlen 150
vocab examples: ['!', '#', '$', '%', '&', "'", "''", "''*Well", "''And", "''Do"]
vocab size 49511
n_train 30277
n_test 225


In [22]:
out_ch = [1, 2, 4, 8, 16]
embd_size = [1, 2, 4, 8, 16, 32, 64, 128]
filters = [[1], [1,3], [1,3,5]]

print(param_accuracy_graph(out_ch,embd_size,filters))


Model 1/120  0.8%


/home/matt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0.42425641417503357, 0.42425641417503357, 0.42425641417503357, 0.6347550749778748, 0.42425641417503357, 0.6347550749778748, 0.42425641417503357, 0.6532206535339355, 0.4985329210758209, 0.42425641417503357, 0.4436337351799011, 0.6532206535339355, 0.42855358123779297, 0.5051319599151611, 0.6532206535339355, 0.49900272488594055, 0.6532206535339355, 0.42425641417503357, 0.42425641417503357, 0.7276718616485596, 0.42425641417503357, 0.5162067413330078, 0.42425641417503357, 0.49900272488594055, 0.42425641417503357, 0.4271731674671173, 0.42425641417503357, 0.7276718616485596, 0.4768827557563782, 0.42425641417503357, 0.49900272488594055, 0.42425641417503357, 0.42425641417503357, 0.42425641417503357, 0.6102472543716431, 0.4451636075973511, 0.6102472543716431, 0.42425641417503357, 0.4436337351799011, 0.42425641417503357, 0.49900272488594055, 0.49900272488594055, 0.42425641417503357, 0.42425641417503357, 0.42425641417503357, 0.42425641417503357, 0.42425641417503357, 0.6102472543716431, 0.42425641

[1.1844031512737274, 1.870581716299057, 1.1003473401069641, 1.8092992305755615, 1.2883502542972565, 1.4416031539440155, 1.4126358032226562, 1.4133673906326294, 1.2169909477233887, 1.2825646698474884, 1.6935737431049347, 1.424175649881363, 1.2460618913173676, 1.3778237700462341, 1.9293204545974731, 1.6220225095748901, 1.888977289199829, 1.2409732341766357, 1.2929057478904724, 2.0380067229270935, 1.5363051295280457, 1.5293291211128235, 1.2540350556373596, 1.6422767341136932, 1.1881908476352692, 1.6296097934246063, 1.1003473401069641, 2.3583008646965027, 1.2030435502529144, 1.1101477444171906, 1.3531244695186615, 1.6347659528255463, 1.1003473401069641, 1.8391595780849457, 1.6328182220458984, 1.3154498636722565, 1.8430787324905396, 1.1846758127212524, 1.1197246611118317, 1.195211410522461, 1.2807645201683044, 1.1752005517482758, 1.1845848560333252, 1.1003473401069641, 1.5503972470760345, 1.5855774283409119, 1.187611997127533, 1.5250943303108215, 1.5866733193397522, 2.2078932523727417, 1.28

[2.1093768775463104, 2.9462768137454987, 1.9517138600349426, 3.130707800388336, 2.168162077665329, 2.5190530717372894, 2.4186697602272034, 2.266692668199539, 2.2980263233184814, 2.286692440509796, 3.0785814225673676, 2.3867655098438263, 2.097428411245346, 2.353506237268448, 3.049582839012146, 2.4903509318828583, 2.864659756422043, 2.0923397541046143, 2.172717571258545, 2.9523046612739563, 2.387671649456024, 2.8236536383628845, 2.105401575565338, 2.6349034011363983, 2.163873314857483, 2.7402903139591217, 1.9517138600349426, 3.328925520181656, 2.178726017475128, 2.02348855137825, 2.20449098944664, 2.9424363672733307, 1.9521308839321136, 2.8483537137508392, 2.9621903896331787, 2.29113233089447, 2.9817235469818115, 2.160358279943466, 1.9710911810398102, 2.0735859274864197, 2.265341728925705, 2.0265670716762543, 2.0359513759613037, 1.9567000269889832, 2.716524213552475, 2.696076989173889, 2.0739122331142426, 2.588357985019684, 2.4380398392677307, 3.574168860912323, 2.136031597852707, 2.7994

...

[22.82191999151837, 95.56252837181091, 23.826414157170802, 94.98951789736748, 33.630947299301624, 25.102894295414444, 84.15429475903511, 21.302117871528026, 26.81490262178704, 25.75856336671859, 97.51930975914001, 23.15694619831629, 22.251711522461846, 27.30006827507168, 97.20098912715912, 34.81877042166889, 89.51926332712173, 19.97583478759043, 34.070156678557396, 96.87416499853134, 39.84051338210702, 30.647574574220926, 26.687826824374497, 79.92669369280338, 26.662213433301076, 90.35204744338989, 17.927349899429828, 97.1824501156807, 20.000505916192196, 21.872081832727417, 33.4992932472378, 93.52664856612682, 22.84098263666965, 94.63279318809509, 50.87701505422592, 26.840886570047587, 92.15502962470055, 41.937212624936365, 19.6333330598427, 24.63650046265684, 78.59541067481041, 18.060353906126693, 23.741179964505136, 19.58741017646389, 78.47762709856033, 94.02384942770004, 22.49755024840124, 18.48269574972801, 22.57268775656121, 92.07683843374252, 35.15693408064544, 97.27109070122242

[22.858743255375884, 97.5623848438263, 23.967834511771798, 96.98940715193748, 33.840709045529366, 25.206805770925712, 86.11296245455742, 21.326413525210228, 26.86019512731582, 25.799888624809682, 99.51901739835739, 23.224742505932227, 22.310199251631275, 27.44053779821843, 99.20093631744385, 34.94021820090711, 91.5174714922905, 20.026940926210955, 34.27690772712231, 98.8734273314476, 40.08117028698325, 30.843211062718183, 26.810267158783972, 81.87862248718739, 26.706374667352065, 92.34138691425323, 17.942280442337506, 99.17712223529816, 20.038363913656212, 21.927379210246727, 33.62836140207946, 95.52452300488949, 22.89706529979594, 96.62974828481674, 51.26075914502144, 26.898940490093082, 94.15488061308861, 42.34887770691421, 19.687527661095373, 24.726704249391332, 80.33447423577309, 18.09083244780777, 23.772884857840836, 19.646529184479732, 80.2923938035965, 96.01732432842255, 22.573704696493223, 18.507642455631867, 22.61290242272662, 94.07551968097687, 35.271411230787635, 99.27074082

[22.888865794055164, 99.561663210392, 24.037639340385795, 98.98938342928886, 34.02329111099243, 25.231357123178896, 88.09294363856316, 21.351518449024297, 26.901443253736943, 25.86506633553654, 101.50036358833313, 23.340222479309887, 22.327451485209167, 27.467437781393528, 101.2009174823761, 35.070345321670175, 93.50489634275436, 20.06164517137222, 34.454807698726654, 100.87289237976074, 40.322892498224974, 30.98885954171419, 26.850913698319346, 83.86624421179295, 26.828158335527405, 94.33439129590988, 17.960617512115277, 101.17711973190308, 20.06655545870308, 21.9620309562888, 33.730183626525104, 97.52050702273846, 22.932102989172563, 98.60471522808075, 52.17006157338619, 26.941805387847126, 96.1544329226017, 42.6299843051238, 19.724032777245156, 24.795768670970574, 82.3052279651165, 18.107687299809186, 23.794036192586645, 19.670535622513853, 82.2809574007988, 98.01732122898102, 22.60539845470339, 18.52658672077814, 22.63377493812004, 95.96865999698639, 35.48933789692819, 101.27073450

[22.896107060369104, 101.56164675951004, 24.047404693905264, 100.98936960101128, 34.10158684104681, 25.242781238106545, 90.09150567650795, 21.354414516477846, 26.918676956556737, 25.88053231034428, 103.5003490447998, 23.34959721332416, 22.334098129998893, 27.492724299430847, 103.20086407661438, 35.203676177188754, 95.49563735723495, 20.067259308998473, 34.51785479299724, 102.87286686897278, 40.46940225735307, 31.014442668529227, 26.861306050326675, 85.83708603680134, 26.838928245473653, 96.32293939590454, 17.964937551529147, 103.17701840400696, 20.07125666900538, 21.971274285577238, 33.83947572577745, 99.51861195266247, 22.94102092925459, 100.60465204715729, 52.78691228479147, 26.953663456020877, 98.15431895852089, 42.70555622212123, 19.731050985050388, 24.80787219456397, 84.26973202824593, 18.109594839741476, 23.800686207483523, 19.67513011203846, 84.27195781469345, 100.01732110977173, 22.621463423711248, 18.529825312638422, 22.63826810842147, 97.96815723180771, 35.53653072565794, 103

[22.92282531457022, 103.55935084819794, 24.068311331328005, 102.98935350775719, 34.21729316934943, 25.3302554054535, 92.08943673968315, 21.381146156578325, 26.949340076185763, 25.93220921512693, 105.50032913684845, 23.37732107611373, 22.37273389333859, 27.52548322826624, 105.20086312294006, 35.24788269586861, 97.49154031276703, 20.08589007204864, 34.63728825561702, 104.87251913547516, 40.55135538801551, 31.04835551721044, 26.895549759734422, 87.80818666517735, 27.198418475221843, 98.319276034832, 17.995300872367807, 105.17701077461243, 20.09504303499125, 22.00706472992897, 33.93079940136522, 101.51076273620129, 22.98499295115471, 102.60391038656235, 52.85567398741841, 26.997955068713054, 100.15332171320915, 42.83302210725378, 19.77426174574066, 24.843713542213663, 86.26663079857826, 18.128036018344574, 23.846536954748444, 19.69954537873855, 86.27174735069275, 102.01731550693512, 22.649510489893146, 18.552360225090524, 22.66872352961218, 99.96515256166458, 35.635834354907274, 105.270719